In [1]:
from audioop import reverse
from collections import defaultdict, namedtuple
import pandas as pd
from collections import defaultdict
import numpy as np
import pprint
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',100)

get_support function:get the support value

In [2]:
def get_support(transactions, item_set):
    match_count = 0
    for transaction in transactions:
        if item_set.issubset(set(transaction)):
            match_count += 1
            
    return float(match_count)/len(transactions)

find_rule:a class that can find rule

In [3]:
class find_rule(object):
    def __init__(self,frequent_dic:dict[set,float],confidence) -> None:
        self.frequent_dic=frequent_dic
        self.confidence=confidence
        self.ansdf=pd.DataFrame({'antecedent':[],'confidence':[],'Lift':[],'PS':[],'coeficient':[]})

    def find(self,itemdata):
        s_rep=get_support(itemdata,{'REPEAT1.0'})
        for frequentset in self.frequent_dic:
            if len(frequentset)<2:
                continue
            s1=self.frequent_dic[frequentset]*s_rep
            antecedent= frequentset-{'REPEAT1.0'}
            s2=get_support(itemdata,antecedent)
            conf=float(s1)/s2
            if conf>self.confidence:
                Lift=conf/s_rep
                PS=s1-s2*s_rep
                coe=(s1-s2*s_rep)/np.sqrt(s_rep*s2*(1-s2)*(1-s_rep))
                new_row={'antecedent':antecedent,'confidence':conf,'Lift':Lift,'PS':PS,'coeficient':coe}
                self.ansdf=self.ansdf.append(new_row, ignore_index=True)

solve:最后集成的一个函数，只使用它即可寻找频繁项集，发掘关联规则

In [4]:
def solve(itemdf_all,sup,conf):
    """
    solve(itemdf_all:Dataframe)->answers
    """
    item_map={
        'EVERE0-1':'ever-repeated',
        'ST001D01T':'international-Grade',
        'ISCEDL':'ISCED level',
        'ST005Q01TA':'mother-school',
        'ST013Q01TA':'books-inroom',
        'REPEAT':'REPEAT'
        }
    for i in range(len(itemdf_all)):
        for col in itemdf_all.columns:
            if itemdf_all[col][i]=='missing':
                itemdf_all[col][i] = None
            else :
                itemdf_all[col][i]= item_map[col]+str(itemdf_all[col][i])

    itemdf=itemdf_all.loc[itemdf_all['REPEAT']=='REPEAT1.0']
    itemdata = itemdf.values.tolist()
    itemdata_all=itemdf_all.values.tolist()
    result = []
    length=len(itemdata)
    for itemset, support in find_frequent_itemsets(itemdata, length*sup):
        if 'REPEAT1.0' in itemset:
            result.append((itemset,support/length))

    result = sorted(result, key=lambda i: i[1],reverse=True)
    frequent_dic={}
    for i in result:
        frequent_dic[frozenset(i[0])]=i[1]
    ans=find_rule(frequent_dic,conf)
    ans.find(itemdata_all)
    return frequent_dic,ans.ansdf

find_frequent_itemsets:寻找频繁项集，用到了FPTREE类

In [5]:
def find_frequent_itemsets(transactions, minimum_support):
    items = defaultdict(lambda: 0) # mapping from items to their supports

    # 计算一项集的支持度
    for transaction in transactions:
        for item in transaction:
            items[item] += 1

    # 去除不频繁的项
    items = dict((item, support) for item, support in items.items()
        if support >= minimum_support)

    # 按照支持度对项排序
    def clean(transaction):
        transaction = list(filter(lambda v: v in items, transaction))
        transaction.sort(key=lambda v: items[v], reverse=True)
        return transaction

    master = FPTREE()
    for transaction in map(clean, transactions):
        master.add(transaction)

    def find_condition_tree(tree, suffix):
        for item, nodes in tree.items():
            support = sum(n.count for n in nodes)
            if support >= minimum_support and item not in suffix:
                found_set = [item] + suffix
                yield (found_set, support) 

                # 建立条件FP树
                cond_tree = conditional_tree_from_paths(tree.prefix_paths(item))
                for s in find_condition_tree(cond_tree, found_set):
                    yield s 

    #迭代返回频繁项集结果
    for itemset in find_condition_tree(master, []):
        yield itemset

FPTREE类：FP树的数据结构实现

In [6]:
class FPTREE(object):
    Route = namedtuple('Route', 'head tail')

    def __init__(self):
        # The root node of the tree.
        self._root = FPNode(self, None, None)

        # A dictionary mapping items to the head and tail of a path of
        # "neighbors" that will hit every node containing that item.
        self._routes = {}
    # 属性语法糖，这次新学的，很有用哦
    @property
    def root(self):
        return self._root

    def add(self, transaction):
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # There is already a node in this tree for the current
                # transaction item; reuse it.
                next_point.increment()
            else:
                # Create a new point and add it as a child of the point we're
                # currently looking at.
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the route of nodes that contain this item to include
                # our new node.
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # First node for this item; start a new route.
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        for item in self._routes.keys():
            yield (item, self.nodes(item))

    def nodes(self, item):
        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
            node = node.neighbor

    def prefix_paths(self, item):
        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        print ('Tree:')
        self.root.inspect(1)

        print ()
        print ('Routes:')
        for item, nodes in self.items():
            print ('  %r' % item)
            for node in nodes:
                print ('    %r' % node)

conditional_tree_from_paths：构建条件FP树

In [7]:
def conditional_tree_from_paths(paths):
    tree = FPTREE()
    condition_item = None
    items = set()

    # Import the nodes in the paths into the new tree. Only the counts of the
    # leaf notes matter; the remaining counts will be reconstructed from the
    # leaf counts.
    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item

        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                # Add a new node to the tree.
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point

    assert condition_item is not None

    # Calculate the counts of the non-leaf nodes.
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count

    return tree

FP树的结点结构

In [8]:
class FPNode(object):
    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None

    def add(self, child):
        """给当前结点添加一个孩子"""

        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        """
        查找该结点是否有孩子是寻找的结点
        """
        try:
            return self._children[item]
        except KeyError:
            return None

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        """结点所在树"""
        return self._tree

    @property
    def item(self):
        return self._item

    @property
    def count(self):
        return self._count

    def increment(self):
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        return self._item is None and self._count is None

    @property
    def leaf(self):
        return len(self._children) == 0

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value

    @property
    def neighbor(self):
        return self._neighbor

    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value

    @property
    def children(self):
        return tuple(self._children.itervalues())

    def inspect(self, depth=0):
        print ('  ' * depth) + repr(self)
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)

主函数

In [9]:
raw_path='df_self.csv'
if __name__ == '__main__':
    itemdf_all = pd.read_csv(raw_path)
    frequent_dic,rule_df=solve(itemdf_all,0.6,0.15)
    frequent_dic
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(frequent_dic)

{   frozenset({'REPEAT1.0'}): 1.0,
    frozenset({'REPEAT1.0', 'ever-repeated1'}): 1.0,
    frozenset({'REPEAT1.0', 'ISCED level2.0995'}): 0.8850442579606851,
    frozenset({'REPEAT1.0', 'ISCED level2.0995', 'ever-repeated1'}): 0.8850442579606851,
    frozenset({'REPEAT1.0', 'international-Grade8.5'}): 0.7535348890677089,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ever-repeated1'}): 0.7535348890677089,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995'}): 0.6777790550638004,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995', 'ever-repeated1'}): 0.6777790550638004}


In [10]:
    rule_df

,antecedent,confidence,Lift,PS,coeficient
0,(ever-repeated1),1.000000,4.839867,0.163927,1.000000
1,(ISCED level2.0995),0.276128,1.336423,0.046033,0.240403
2,"(ISCED level2.0995, ever-repeated1)",1.000000,4.839867,0.145082,0.926994
3,(international-Grade8.5),0.849203,4.110031,0.117812,0.751995
4,"(international-Grade8.5, ever-repeated1)",1.000000,4.839867,0.123524,0.841478
5,"(ISCED level2.0995, international-Grade8.5)",0.878689,4.252735,0.107111,0.722769
6,"(ISCED level2.0995, international-Grade8.5, ever-repeated1)",1.000000,4.839867,0.111106,0.790763


分国家验证辛普森悖论

In [11]:
from IPython.display import display
rule_df_list=[]
for i in [152,188,214,484,591,724]:
    print('国家'+str(i)+':')
    itemdf_all = pd.read_csv('df_'+str(i)+'.csv')
    frequent_dic,rule_df=solve(itemdf_all,0.6,0.15)
#     rule_df_list.append(rule_df)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(frequent_dic)   
    display(rule_df)

国家152:
{   frozenset({'REPEAT1.0'}): 1.0,
    frozenset({'REPEAT1.0', 'ever-repeated1'}): 1.0,
    frozenset({'ISCED level2.9', 'REPEAT1.0'}): 0.8636363636363636,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'ever-repeated1'}): 0.8636363636363636,
    frozenset({'REPEAT1.0', 'international-Grade8.5'}): 0.7305986696230599,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ever-repeated1'}): 0.7305986696230599,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'international-Grade8.5'}): 0.7305986696230599,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'international-Grade8.5', 'ever-repeated1'}): 0.7305986696230599,
    frozenset({'REPEAT1.0', 'books-inroom1.4975'}): 0.6751662971175166,
    frozenset({'REPEAT1.0', 'books-inroom1.4975', 'ever-repeated1'}): 0.6751662971175166}


,antecedent,confidence,Lift,PS,coeficient
0,(ever-repeated1),1.000000,6.279379,0.133890,1.000000
1,"(ISCED level2.9, ever-repeated1)",1.000000,6.279379,0.115633,0.917546
2,(international-Grade8.5),0.653122,4.101200,0.087979,0.628383
3,"(international-Grade8.5, ever-repeated1)",1.000000,6.279379,0.097820,0.833743
4,"(ISCED level2.9, international-Grade8.5)",0.653122,4.101200,0.087979,0.628383
5,"(ISCED level2.9, international-Grade8.5, ever-repeated1)",1.000000,6.279379,0.097820,0.833743
6,(books-inroom1.4975),0.216803,1.361389,0.028542,0.156011
7,"(ever-repeated1, books-inroom1.4975)",1.000000,6.279379,0.090398,0.797516


国家188:
{}


,antecedent,confidence,Lift,PS,coeficient


国家214:
{   frozenset({'REPEAT1.0'}): 1.0,
    frozenset({'REPEAT1.0', 'ever-repeated1'}): 1.0,
    frozenset({'international-Grade9.5', 'REPEAT1.0'}): 0.75,
    frozenset({'international-Grade9.5', 'REPEAT1.0', 'ever-repeated1'}): 0.75,
    frozenset({'ISCED level2.9', 'REPEAT1.0'}): 0.75,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'ever-repeated1'}): 0.75,
    frozenset({'international-Grade9.5', 'ISCED level2.9', 'REPEAT1.0'}): 0.75,
    frozenset({'international-Grade9.5', 'ISCED level2.9', 'REPEAT1.0', 'ever-repeated1'}): 0.75,
    frozenset({'REPEAT1.0', 'books-inroom1.4975'}): 0.75,
    frozenset({'REPEAT1.0', 'books-inroom1.4975', 'ever-repeated1'}): 0.75,
    frozenset({'international-Grade9.5', 'REPEAT1.0', 'books-inroom1.4975'}): 0.625,
    frozenset({'international-Grade9.5', 'REPEAT1.0', 'books-inroom1.4975', 'ever-repeated1'}): 0.625,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'books-inroom1.4975'}): 0.625,
    frozenset({'ISCED level2.9', 'REPEAT1.0', 'books-inroom1

...

,antecedent,confidence,Lift,PS,coeficient
0,(ever-repeated1),1.000000,11.403131,0.080005,1.000000
1,(ISCED level2.0995),0.622928,7.103332,0.066502,0.712757
2,"(ISCED level2.0995, ever-repeated1)",1.000000,11.403131,0.070611,0.934202
3,"(ever-repeated1, books-inroom1.4975)",1.000000,11.403131,0.058555,0.844695
4,(international-Grade8.5),0.588816,6.714344,0.052288,0.604703
5,"(international-Grade8.5, ever-repeated1)",1.000000,11.403131,0.056050,0.825220
6,"(ISCED level2.0995, international-Grade8.5)",0.588816,6.714344,0.052288,0.604703
7,"(ISCED level2.0995, international-Grade8.5, ever-repeated1)",1.000000,11.403131,0.056050,0.825220
8,"(ISCED level2.0995, books-inroom1.4975)",0.702929,8.015590,0.050469,0.650219
9,"(ISCED level2.0995, ever-repeated1, books-inroom1.4975)",1.000000,11.403131,0.052606,0.797859


国家591:
{   frozenset({'REPEAT1.0'}): 1.0,
    frozenset({'REPEAT1.0', 'ever-repeated1'}): 1.0,
    frozenset({'REPEAT1.0', 'ISCED level2.0995'}): 0.8442871587462083,
    frozenset({'REPEAT1.0', 'ISCED level2.0995', 'ever-repeated1'}): 0.8442871587462083,
    frozenset({'REPEAT1.0', 'books-inroom1.4975'}): 0.7522750252780587,
    frozenset({'REPEAT1.0', 'books-inroom1.4975', 'ever-repeated1'}): 0.7522750252780587,
    frozenset({'REPEAT1.0', 'books-inroom1.4975', 'ISCED level2.0995'}): 0.647118301314459,
    frozenset({'books-inroom1.4975', 'REPEAT1.0', 'ISCED level2.0995', 'ever-repeated1'}): 0.647118301314459,
    frozenset({'REPEAT1.0', 'international-Grade8.5'}): 0.6117290192113246,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ever-repeated1'}): 0.6117290192113246,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995'}): 0.6117290192113246,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995', 'ever-repeated1'}): 0.6117290192113246}

,antecedent,confidence,Lift,PS,coeficient
0,(ever-repeated1),1.000000,4.614762,0.169739,1.000000
1,(ISCED level2.0995),0.764652,3.528687,0.131106,0.745893
2,"(ISCED level2.0995, ever-repeated1)",1.000000,4.614762,0.143308,0.899678
3,(books-inroom1.4975),0.234700,1.083086,0.012505,0.065900
4,"(ever-repeated1, books-inroom1.4975)",1.000000,4.614762,0.127690,0.839063
5,"(ISCED level2.0995, books-inroom1.4975)",0.765550,3.532832,0.100535,0.630858
6,"(ISCED level2.0995, ever-repeated1, books-inroom1.4975)",1.000000,4.614762,0.109841,0.767831
7,(international-Grade8.5),0.732446,3.380062,0.093341,0.588462
8,"(international-Grade8.5, ever-repeated1)",1.000000,4.614762,0.103834,0.743233
9,"(ISCED level2.0995, international-Grade8.5)",0.732446,3.380062,0.093341,0.588462


国家724:
{   frozenset({'REPEAT1.0'}): 1.0,
    frozenset({'REPEAT1.0', 'ever-repeated1'}): 1.0,
    frozenset({'REPEAT1.0', 'ISCED level2.0995'}): 0.9998409922086182,
    frozenset({'REPEAT1.0', 'ISCED level2.0995', 'ever-repeated1'}): 0.9998409922086182,
    frozenset({'REPEAT1.0', 'international-Grade8.5'}): 0.7843854348863094,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ever-repeated1'}): 0.7843854348863094,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995'}): 0.7843854348863094,
    frozenset({'REPEAT1.0', 'international-Grade8.5', 'ISCED level2.0995', 'ever-repeated1'}): 0.7843854348863094}


,antecedent,confidence,Lift,PS,coeficient
0,(ever-repeated1),1.000000,4.124980,0.183655,1.000000
1,(ISCED level2.0995),0.242527,1.000419,0.000102,0.009865
2,"(ISCED level2.0995, ever-repeated1)",1.000000,4.124980,0.183626,0.999895
3,(international-Grade8.5),0.935876,3.860468,0.140898,0.817108
4,"(international-Grade8.5, ever-repeated1)",1.000000,4.124980,0.144057,0.856597
5,"(ISCED level2.0995, international-Grade8.5)",0.935876,3.860468,0.140898,0.817108
6,"(ISCED level2.0995, international-Grade8.5, ever-repeated1)",1.000000,4.124980,0.144057,0.856597
